# Trading Pipeline Demo

Demonstrates running the pipeline for a date range and a single day.

In [1]:
import sys, os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


In [2]:
from trading_bot.agents.llm_roles import (
    MarketAnalystAgent,
    NewsSummarizerAgent,
    StrategyTypeAgent, 
    IndicatorSelectionAgent,
    StrategyPlannerAgent,
)

from trading_bot.coordinator import Coordinator
from trading_bot.pipeline import Pipeline
from trading_bot.storage import JSONStorage
from trading_bot.backtest import Backtester
from trading_bot.portfolio import Portfolio
import yfinance as yf

coord = Coordinator([
    MarketAnalystAgent(),
    NewsSummarizerAgent(),
    StrategyTypeAgent(), 
    IndicatorSelectionAgent(),
    StrategyPlannerAgent(),
])

storage = JSONStorage()
backtester = Backtester()
portfolio = Portfolio()
pipe = Pipeline(coord, storage=storage, backtester=backtester, portfolio=portfolio)


In [3]:
single_result = pipe.run('TSLA', today='2025-02-04')
single_result['strategy'], single_result['files']


/Users/max.chen/Python_Code/AI-Trading-Agent/trading_bot/pipeline.py:111: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(price_series.iloc[0])


({'agent': 'StrategyPlannerAgent',
  'symbol': 'TSLA',
  'date': '2025-02-04',
  'strategy_type': 'reversal',
  'summary': "Potential reversal in TSLA's downtrend",
  'rationale': "Despite negative news sentiment and disappointing Q4 earnings, TSLA's fundamentals remain strong and macroeconomic conditions are favourable. The recent sell-off may be overdone, suggesting a possible reversal.",
  'setup': {'direction': 'long',
   'entry_price': 385.0,
   'entry_reason': 'Anticipated reversal point',
   'stop_loss': {'price': 374.39, 'reason': 'Recent low'},
   'capital_risk_pct': 2.75,
   'targets': [{'price': 395.0,
     'size_pct': 50.0,
     'reason': 'Near term resistance'},
    {'price': 405.0, 'size_pct': 50.0, 'reason': 'Recent high'}],
   'trailing_stop': {'method': 'ATR',
    'period': 14,
    'description': 'Trailing stop based on Average True Range',
    'reason': 'To protect profits and limit downside risk'}},
  'derivations': {'session_range': {'high': 389.17, 'low': 374.39},


In [4]:
single_result = pipe.run('TSLA', today='2025-03-04')
single_result['strategy'], single_result['files']

/Users/max.chen/Python_Code/AI-Trading-Agent/trading_bot/pipeline.py:111: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(price_series.iloc[0])


({'agent': 'StrategyPlannerAgent',
  'symbol': 'TSLA',
  'date': '2025-03-04',
  'strategy_type': 'pullback',
  'summary': 'Buying pullback in the context of strong fundamentals',
  'rationale': 'Despite the recent price drop, the strong fundamentals and positive macroeconomic conditions suggest this could be a temporary pullback. The RSI is nearing oversold conditions and the EMA is still positive indicating potential for a price rebound.',
  'setup': {'direction': 'long',
   'entry_price': 284.78,
   'entry_reason': 'RSI nearing oversold conditions and EMA still positive',
   'stop_loss': {'price': 273.6, 'reason': 'Previous session low'},
   'capital_risk_pct': 5,
   'targets': [{'price': 300.73,
     'size_pct': 50,
     'reason': 'Recent swing high'},
    {'price': 303.94, 'size_pct': 50, 'reason': 'Recent swing high'}],
   'trailing_stop': {'method': 'ATR',
    'period': 14,
    'description': 'Trailing stop using ATR method',
    'reason': 'To protect profits and limit downside 

In [ ]:
range_result = pipe.run('TSLA', start_date='2025-02-03', end_date='2025-02-05')
range_result['files'], range_result['backtest']

In [ ]:
portfolio.total_pnl(), [p.__dict__ for p in portfolio.closed_positions()]


In [ ]:
from trading_bot.openai_client import call_llm
print(call_llm("Say hello in JSON format"))


In [ ]:
df = yf.download('TSLA', start='2025-02-03', end='2025-02-05')


In [ ]:
import yfinance as yf
import pandas as pd
end = pd.to_datetime("2025-02-04")
start = end - pd.Timedelta(days=7)
df = yf.download(
    "TSLA",
    start=start.strftime("%Y-%m-%d"),
    end=end.strftime("%Y-%m-%d"),
    interval="1h",
    auto_adjust=True,
    progress=False,
)
df

In [ ]:
import yfinance as yf
import pandas as pd
end = pd.to_datetime("2025-03-04")
start = end - pd.Timedelta(days=7)
df = yf.download(
    "TSLA",
    start=start.strftime("%Y-%m-%d"),
    end=end.strftime("%Y-%m-%d"),
    interval="1h",
    auto_adjust=True,
    progress=False,
)
df